In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from tqdm import tqdm

# SUUMOから物件情報を取得する関数
def scrape_suumo(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    contents = soup.find_all('div', class_='cassetteitem')
    data_list = []

    for content in contents:
        detail = content.find('div', class_='cassetteitem-detail')
        table = content.find('table', class_='cassetteitem_other')

        title = detail.find('div', class_='cassetteitem_content-title').text
        address = detail.find('li', class_='cassetteitem_detail-col1').text
        access = detail.find('li', class_='cassetteitem_detail-col2').text
        age_text = detail.find('li', class_='cassetteitem_detail-col3').text
        age_match = re.search(r'\d+', age_text)
        age = int(age_match.group()) if age_match else None

        tr_tags = table.find_all('tr', class_='js-cassette_link')

        for tr_tag in tr_tags:
            floor_text = re.search(r'\d+', tr_tag.find_all("td")[2].text)
            floor = int(floor_text.group()) if floor_text else None

            price, first_fee, capacity = tr_tag.find_all("td")[3:6]
            fee, management_fee = price.find_all('li')
            deposit, qratuity = first_fee.find_all('li')
            madori, menseki = capacity.find_all('li')

            cost_text = re.search(r'\d+', fee.text)
            cost = int(cost_text.group()) if cost_text else None

            size_text = re.search(r'\d+', menseki.text)
            size = int(size_text.group()) if size_text else None

            d = {
                "title": title,
                "address": address,
                "access": access,
                "age": age,
                "floor": floor,
                "cost": cost,
                "management_fee": management_fee.text,
                "deposit": deposit.text,
                "qratuity": qratuity.text,
                "layout": madori.text,
                "size": size,
            }

            data_list.append(d)

    return data_list

# メインの処理
url_template = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&sc=13102&sc=13103&sc=13104&sc=13113&ta=13&cb=0.0&ct=30.0&md=06&md=07&md=09&md=10&md=11&md=13&md=14&ts=1&et=9999999&mb=0&mt=9999999&cn=9999999&fw2='
last_page = int(BeautifulSoup(requests.get(url_template.format(1)).text, 'html.parser').find('ol', class_='pagination-parts').find_all('li')[-1].text)

# 全ページのデータを取得
all_data = []
for i in tqdm(range(1, last_page+1)):
    target_url = url_template.format(i)
    data_list = scrape_suumo(target_url)
    all_data.extend(data_list)
    sleep(1)

# データをデータフレームに変換
df_all_data = pd.DataFrame(all_data)

# address、age、floor、costが同じ場合に重複とみなし、重複を排除
df_no_duplicates = df_all_data.drop_duplicates(subset=['address', 'age', 'floor', 'cost'])

# 住所から区を抽出して新しい列 'Area' を作成
def extract_area(address):
    match = re.search(r'東京都(.*?)区', address)
    if match:
        return match.group(1) + "区"
    return None

df_no_duplicates['Area'] = df_no_duplicates['address'].apply(extract_area)

# CSVファイルに保存（Shift-JISエンコーディング）
df_no_duplicates.to_csv('sumodb6.csv', index=False, encoding='shift_jis')

# CSVファイルからデータを読み込み
df = pd.read_csv('sumodb6.csv', encoding='shift_jis')

# "cost" 列を数値列に変換
df['cost'] = pd.to_numeric(df['cost'], errors='coerce')

# 変換に成功したかどうかを確認
is_numeric = pd.notna(df['cost'])

# データフレームの最初の10行を表示
print(df.head(10))


100%|██████████| 57/57 [04:03<00:00,  4.28s/it]

             title    address  \
0       リューズ・リバー月島  東京都中央区月島４   
1       リューズ・リバー月島  東京都中央区月島４   
2       リューズ・リバー月島  東京都中央区月島４   
3       リューズ・リバー月島  東京都中央区月島４   
4  ザ・ライオンズフォーシア八丁堀   東京都中央区湊１   
5  ザ・ライオンズフォーシア八丁堀   東京都中央区湊１   
6  ザ・ライオンズフォーシア八丁堀   東京都中央区湊１   
7  ザ・ライオンズフォーシア八丁堀   東京都中央区湊１   
8  ザ・ライオンズフォーシア八丁堀   東京都中央区湊１   
9  ザ・ライオンズフォーシア八丁堀   東京都中央区湊１   

                                              access  age  floor  cost  \
0  \n都営大江戸線/勝どき駅 歩6分\n東京メトロ有楽町線/月島駅 歩8分\n東京メトロ日比谷...   26      4    17   
1  \n都営大江戸線/勝どき駅 歩6分\n東京メトロ有楽町線/月島駅 歩8分\n東京メトロ日比谷...   26      5    20   
2  \n都営大江戸線/勝どき駅 歩6分\n東京メトロ有楽町線/月島駅 歩8分\n東京メトロ日比谷...   26      5    21   
3  \n都営大江戸線/勝どき駅 歩6分\n東京メトロ有楽町線/月島駅 歩8分\n東京メトロ日比谷...   26      3    21   
4  \nＪＲ京葉線/八丁堀駅 歩5分\n東京メトロ有楽町線/新富町駅 歩9分\n東京メトロ日比谷...   10      2    20   
5  \nＪＲ京葉線/八丁堀駅 歩5分\n東京メトロ有楽町線/新富町駅 歩9分\n東京メトロ日比谷...   10      3    21   
6  \nＪＲ京葉線/八丁堀駅 歩5分\n東京メトロ有楽町線/新富町駅 歩9分\n東京メトロ日比谷...   10      4    21   
7  \nＪＲ京葉線/八丁堀駅 歩5分\n東京メトロ有楽町線/新富町駅 歩9分\n東京メ


C:\Users\admin\AppData\Local\Temp\ipykernel_15100\303195129.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['Area'] = df_no_duplicates['address'].apply(extract_area)


In [5]:
# CSVファイルにShift-JISでエンコードして保存
df_no_duplicates.to_csv('sumodb6.csv', index=False, encoding='shift_jis')

In [11]:
import sqlite3
import pandas as pd

# SQLiteデータベースに接続
conn = sqlite3.connect('sumodb6.db')

# CSVファイルからデータを読み込む
csv_file = 'sumodb6.csv'
df = pd.read_csv(csv_file, encoding='shift_jis')

# データフレームをSQLiteデータベースに書き込む
df.to_sql('sumodb6_table', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()

print('Database created successfully.')



Database created successfully.


71